### Problem 8 --- Handout 5

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from numba import njit
from scipy.stats import norm

In [2]:
def draw_sample(size, scale=1, seed=0):
    """Draws random exponential samples.

    Args:
        size (tuple): Size of sample.
        scale (float): Expected value of distribution.
        seed (int): Random number generator seed.

    Returns:
        sample (np.ndarray): Samples of size ``size``.

    """
    np.random.seed(seed)
    sample = np.random.exponential(scale, size)
    return sample

In [3]:
def sample_mean(sample, size):
    """Compute row means for 2d array.

    Args:
        sample (np.ndarray): 2d array of shape (n, m).
        size (int): Number of columns of sample (m).

    Returns:
        mean: (np.ndarray): Row means array of size (n).

    """
    mean = np.einsum("ij->i", sample) / size
    return mean

In [4]:
def sample_variance(sample, size, mean=None):
    """Compute sample variance for each row of array.

    Args:
        sample (np.ndarray): 2d array of shape (n, m).
        size (int): Number of columns of sample (m).
        mean (np.ndarrary): Pre-computed mean. Defaults to None
            in which case the mean is computed internally.

    Returns:
        variance (np.ndarray): Row variances array of size (n).

    """
    mean = sample_mean(sample, size) if mean is None else mean
    variance = sample_mean(sample ** 2, size) - mean ** 2
    variance = size * variance / (size - 1)
    return variance

In [5]:
def confidence_interval(mean, variance, size, alpha):
    """Compute confidence interval.

    Args:
        mean (np.ndarray): Mean array of size (n,).
        variance (np.ndarray): Variance array of size (n,).
        size (int): Number of samples (n).
        alpha (float): Confidence parameter.

    Returns:
        left (np.ndarray): Left bounds array of size (n,).
        right (np.ndarray): Right bounds array of size (n,).

    """
    quantile = norm.ppf(1 - alpha / 2)
    factor = quantile / np.sqrt(size)
    scaled_variance = factor * variance
    left = mean - scaled_variance
    right = mean + scaled_variance
    return left, right

In [6]:
def simulation(runs=100_000, size=50, alpha=0.05, scale=1, seed=0):
    """Monte Carlo simulation.

     Computes frequency of times when true mean parameter of
     distribution was contained in estimated confidence interval.

    Args:
        runs (int): Number of Monte Carlo runs.
        size (int): Size of random sample in each run.
        alpha (float): Confidence parameter.
        scale (float): Expected value of distribution from which we
            sample in the simulation.
        seed (int): Random number generator seed.

    Returns:
        coverage (float): True coverage frequency.

    """
    sample = draw_sample((runs, size), scale, seed)
    mean = sample_mean(sample, size)
    variance = sample_variance(sample, size, mean)

    left, right = confidence_interval(mean, variance, size, alpha)

    covered = (left < scale) & (scale < right)
    coverage = covered.mean()
    return coverage

In [7]:
simulation(runs=20_000, size=50, alpha=0.05, scale=1, seed=0)

0.8937

In [8]:
simulation(runs=20_000, size=1_000, alpha=0.05, scale=1, seed=0)

0.94595